In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Install Ultralytics
!pip install ultralytics

# Verify dataset structure
print("Listing /kaggle/input:")
!ls /kaggle/input
print("\nListing /kaggle/input/tennis-object-detection:")
!ls /kaggle/input/tennis-object-detection
print("\nListing /kaggle/input/tennis-object-detection/tennis-object-detection:")
!ls /kaggle/input/tennis-object-detection/tennis-object-detection
print("\nListing /kaggle/input/tennis-object-detection/tennis-object-detection/train:")
!ls /kaggle/input/tennis-object-detection/tennis-object-detection/train
print("\nListing /kaggle/input/tennis-object-detection/tennis-object-detection/valid:")
!ls /kaggle/input/tennis-object-detection/tennis-object-detection/valid

# Copy data.yaml to a writable directory and update it
import os
import shutil

# Copy data.yaml to /kaggle/working/
shutil.copy("/kaggle/input/tennis-object-detection/tennis-object-detection/data.yaml", "/kaggle/working/data.yaml")

# Update data.yaml with the correct paths
yaml_content = """train: /kaggle/input/tennis-object-detection/tennis-object-detection/train/images
val: /kaggle/input/tennis-object-detection/tennis-object-detection/valid/images
test: /kaggle/input/tennis-object-detection/tennis-object-detection/test/images
nc: 2
names: ['ball', 'player']
"""
with open("/kaggle/working/data.yaml", "w") as f:
    f.write(yaml_content)

# Step 1: Train YOLOv8
from ultralytics import YOLO

# Initialize and train the model
model = YOLO("yolov8s.pt")
model.train(
    data="/kaggle/working/data.yaml",  # Use the updated data.yaml from /kaggle/working/
    epochs=30,
    imgsz=1920,  # Matches 1920x1920 images
    batch=4,     # Smaller batch for larger images
    name="tennis_player_model",
    patience=10,
    device=0
)

# Step 2: Save model weights as .pkl
import pickle
import torch

# Extract model state dictionary
model_state_dict = model.model.state_dict()
# Save to .pkl
with open("/kaggle/working/tennis_player_model_weights.pkl", "wb") as f:
    pickle.dump(model_state_dict, f)

print("Model weights saved as /kaggle/working/tennis_player_model_weights.pkl")

# Step 3: Load weights from .pkl and validate
# Initialize a new YOLO model with the same architecture
new_model = YOLO("yolov8s.pt")
# Load the saved state dictionary
with open("/kaggle/working/tennis_player_model_weights.pkl", "rb") as f:
    loaded_state_dict = pickle.load(f)

# Load the weights into the new model
new_model.model.load_state_dict(loaded_state_dict)
new_model.model.eval()  # Set to evaluation mode

# Step 4: Validate using the loaded weights
val_results = new_model.val(data="/kaggle/working/data.yaml", split='valid')

# Step 5: Calculate Accuracy, Precision, Recall, and F1 Score
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

# Extract metrics from validation results
precision = val_results.results_dict['metrics/precision(B)']
recall = val_results.results_dict['metrics/recall(B)']
mAP50 = val_results.results_dict['metrics/mAP50(B)']

# Compute F1 score
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Approximate accuracy for object detection
# Using precision and recall as proxies (not exact, but a common approximation)
if precision > 0 and recall > 0:
    accuracy = (precision * recall) / (precision + recall - (precision * recall))
else:
    accuracy = 0

print(f"Validation Metrics (using loaded .pkl weights):")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"mAP@50: {mAP50:.4f}")
print(f"Approximate Accuracy: {accuracy:.4f}")

# Test on an image (optional)
results = new_model.predict("/kaggle/input/tennis-object-detection/tennis-object-detection/valid/images/image_3201.jpg")
results[0].save("/kaggle/working/output.jpg")
print("Test image saved as /kaggle/working/output.jpg")

# Download model weights
from IPython.display import FileLink
FileLink(r'/kaggle/working/tennis_player_model_weights.pkl')

Listing /kaggle/input:
tennis-object-detection

Listing /kaggle/input/tennis-object-detection:
tennis-object-detection

Listing /kaggle/input/tennis-object-detection/tennis-object-detection:
data.yaml  test  train	valid

Listing /kaggle/input/tennis-object-detection/tennis-object-detection/train:
images	labels

Listing /kaggle/input/tennis-object-detection/tennis-object-detection/valid:
images	labels
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:00<00:00, 113MB/s] 


Ultralytics 8.3.128 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1920, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=tennis_player_model, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, po

100%|██████████| 755k/755k [00:00<00:00, 22.1MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      18/30      7.98G      1.162     0.5867      1.172         21       1920: 100%|██████████| 566/566 [06:19<00:00,  1.49it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:07<00:00,  2.84it/s]

                   all        171        501      0.878      0.882      0.879       0.51



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      19/30      7.99G      1.092     0.5313      1.139         30       1920:   2%|▏         | 9/566 [00:06<06:14,  1.49it/s]

In [4]:
# Install Ultralytics
!pip install ultralytics

# Verify dataset structure
print("Listing /kaggle/input:")
!ls /kaggle/input
print("\nListing /kaggle/input/tennis-object-detection:")
!ls /kaggle/input/tennis-object-detection
print("\nListing /kaggle/input/tennis-object-detection/tennis-object-detection:")
!ls /kaggle/input/tennis-object-detection/tennis-object-detection
print("\nListing /kaggle/input/tennis-object-detection/tennis-object-detection/train:")
!ls /kaggle/input/tennis-object-detection/tennis-object-detection/train
print("\nListing /kaggle/input/tennis-object-detection/tennis-object-detection/valid:")
!ls /kaggle/input/tennis-object-detection/tennis-object-detection/valid

# Copy data.yaml to a writable directory and update it
import os
import shutil

# Copy data.yaml to /kaggle/working/
shutil.copy("/kaggle/input/tennis-object-detection/tennis-object-detection/data.yaml", "/kaggle/working/data.yaml")

# Update data.yaml with the correct paths
yaml_content = """train: /kaggle/input/tennis-object-detection/tennis-object-detection/train/images
val: /kaggle/input/tennis-object-detection/tennis-object-detection/valid/images
test: /kaggle/input/tennis-object-detection/tennis-object-detection/test/images
nc: 2
names: ['ball', 'player']
"""
with open("/kaggle/working/data.yaml", "w") as f:
    f.write(yaml_content)

# Step 1: Load the existing best model weights and save as .pkl
from ultralytics import YOLO
import pickle
import torch

# Load the existing best model weights
# Adjust the path if your best.pt is in a different location
best_model_path = "/kaggle/working/runs/detect/tennis_player_model/weights/best.pt"
best_model = YOLO(best_model_path)
print(f"Best model loaded from {best_model_path}")

# Extract the best model state dictionary
best_model_state_dict = best_model.model.state_dict()

# Save the best weights to .pkl
with open("/kaggle/working/tennis_player_model_best_weights.pkl", "wb") as f:
    pickle.dump(best_model_state_dict, f)

print("Best model weights saved as /kaggle/working/tennis_player_model_best_weights.pkl")

# Step 2: Load weights from .pkl and validate
# Initialize a new YOLO model directly from best.pt (already configured for nc=2)
new_model = YOLO(best_model_path)  # Load from best.pt to avoid configuration mismatch

# Load the saved state dictionary from .pkl
with open("/kaggle/working/tennis_player_model_best_weights.pkl", "rb") as f:
    loaded_state_dict = pickle.load(f)

# Load the weights into the new model
new_model.model.load_state_dict(loaded_state_dict)
new_model.model.eval()  # Set to evaluation mode

# Step 3: Validate using the loaded weights
# Use split='val' to match the 'val' key in data.yaml
val_results = new_model.val(data="/kaggle/working/data.yaml", split='val', imgsz=1280)

# Step 4: Calculate Accuracy, Precision, Recall, and F1 Score
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

# Extract metrics from validation results
precision = val_results.results_dict['metrics/precision(B)']
recall = val_results.results_dict['metrics/recall(B)']
mAP50 = val_results.results_dict['metrics/mAP50(B)']

# Compute F1 score
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Approximate accuracy for object detection
if precision > 0 and recall > 0:
    accuracy = (precision * recall) / (precision + recall - (precision * recall))
else:
    accuracy = 0

print(f"Validation Metrics (using loaded .pkl weights):")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"mAP@50: {mAP50:.4f}")
print(f"Approximate Accuracy: {accuracy:.4f}")

# Test on an image (optional)
results = new_model.predict("/kaggle/input/tennis-object-detection/tennis-object-detection/valid/images/frame_0_jpg.rf.ed5b1f9e28141c48af9b1109c25b235f.jpg", imgsz=1280)
results[0].save("/kaggle/working/output.jpg")
print("Test image saved as /kaggle/working/output.jpg")

# Download model weights
from IPython.display import FileLink
FileLink(r'/kaggle/working/runs/train/tennis_player_model/weights/best.pt')
FileLink(r'/kaggle/working/tennis_player_model_best_weights.pkl')

Listing /kaggle/input:
tennis-object-detection

Listing /kaggle/input/tennis-object-detection:
tennis-object-detection

Listing /kaggle/input/tennis-object-detection/tennis-object-detection:
data.yaml  test  train	valid

Listing /kaggle/input/tennis-object-detection/tennis-object-detection/train:
images	labels

Listing /kaggle/input/tennis-object-detection/tennis-object-detection/valid:
images	labels
Best model loaded from /kaggle/working/runs/detect/tennis_player_model/weights/best.pt
Best model weights saved as /kaggle/working/tennis_player_model_best_weights.pkl
Ultralytics 8.3.128 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.5±0.0 ms, read: 349.0±188.3 MB/s, size: 226.9 KB)


val: Scanning /kaggle/input/tennis-object-detection/tennis-object-detection/valid/labels... 171 images, 0 backgrounds, 0 corrupt: 100%|██████████| 171/171 [00:00<00:00, 830.06it/s]

WARNING ⚠️ val: Cache directory /kaggle/input/tennis-object-detection/tennis-object-detection/valid is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:08<00:00,  1.30it/s]


                   all        171        501      0.897      0.868      0.872      0.519
                  ball        154        158      0.855      0.753      0.759      0.299
                player        171        343      0.939      0.983      0.985      0.739


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 7.3ms preprocess, 18.0ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/val3
Validation Metrics (using loaded .pkl weights):
Precision: 0.8970
Recall: 0.8678
F1 Score: 0.8822
mAP@50: 0.8721
Approximate Accuracy: 0.7892

image 1/1 /kaggle/input/tennis-object-detection/tennis-object-detection/valid/images/frame_0_jpg.rf.ed5b1f9e28141c48af9b1109c25b235f.jpg: 1280x1280 3 players, 18.4ms
Speed: 12.0ms preprocess, 18.4ms inference, 4.5ms postprocess per image at shape (1, 3, 1280, 1280)
Test image saved as /kaggle/working/output.jpg


/kaggle/working/tennis_player_model_best_weights.pkl